In [1]:
import pandas as pd
import numpy as np
import orquest_csv_utils as ocu

utils imported


In [2]:

'''
 The goal of this notebook is to
 provide a matrix displaying the
 number of days per presence types
 and employee.
'''


'\n The goal of this notebook is to\n provide a matrix displaying the\n number of days per presence types\n and employee.\n'

In [3]:
# Load relevant data
work_log = pd.read_csv('hours_associations_incidences.csv')

In [4]:
# Tidy up absence_type
work_log['absence_type'] = work_log['absence_type'].fillna('no_ausente')

In [5]:
'''
 Before creating our matrix, we need to 
 change the granularity of this table to 'day'.
 The first step for this is to sum hours per
 day and person_id.
'''
work_log['planned_hours_day'] = work_log.groupby(['day','person_id'])['planned_hours'].transform('sum')
work_log['worked_hours_day'] = work_log.groupby(['day','person_id'])['worked_hours'].transform('sum')

# Now, we select unique rows ignoring hour in order to get distinct days
work_calendar = work_log[['day','person_id','planned_hours','store_id', 'absence_type']].sort_values(by=['person_id','day']).drop_duplicates(subset=['day','person_id'])

In [6]:
work_log

,day,hour,person_id,worked_hours,asoc_from_date,asoc_to_date,store_id,absence_type,inc_from_date,inc_to_date,was_absent,planned_hours,date_time,planned_hours_day,worked_hours_day
0,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1,no_ausente,NaN,NaN,False,1.0,2023-11-02 06:00:00,7.5,7.5
1,2023-11-02,6,665948,1.0,2008-10-01,2024-04-26,1,no_ausente,NaN,NaN,False,1.0,2023-11-02 06:00:00,7.5,7.5
2,2023-11-02,6,665954,1.0,2020-10-01,2024-04-26,1,no_ausente,NaN,NaN,False,1.0,2023-11-02 06:00:00,5.5,5.5
3,2023-11-02,6,665986,1.0,2008-10-01,2024-04-26,1,no_ausente,NaN,NaN,False,1.0,2023-11-02 06:00:00,6.5,6.5
4,2023-11-02,6,999012,1.0,2022-10-12,2024-04-26,1,no_ausente,NaN,NaN,False,1.0,2023-11-02 06:00:00,6.5,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14111,2024-01-20,22,665923,0.5,2008-10-01,2024-04-26,2,no_ausente,NaN,NaN,False,0.5,2024-01-20 22:00:00,5.5,5.5
14112,2024-01-20,22,665926,0.5,2008-10-01,2024-04-26,2,no_ausente,NaN,NaN,False,0.5,2024-01-20 22:00:00,6.5,6.5
14113,2024-01-20,22,665979,0.5,2023-07-01,2024-04-26,2,no_ausente,NaN,NaN,False,0.5,2024-01-20 22:00:00,7.5,7.5
14114,2024-01-20,22,665997,0.0,2023-07-01,2024-04-26,2,Enfermedad común,2024-01-12,2024-02-17,True,0.5,2024-01-20 22:00:00,4.5,0.0


In [7]:
work_calendar

,day,person_id,planned_hours,store_id,absence_type
0,2023-11-02,665917,1.0,1,no_ausente
107,2023-11-03,665917,1.0,1,no_ausente
230,2023-11-04,665917,1.0,1,no_ausente
392,2023-11-06,665917,1.0,1,no_ausente
517,2023-11-07,665917,1.0,1,no_ausente
...,...,...,...,...,...
6859,2024-01-13,1783676,1.0,1,no_ausente
7457,2024-01-19,1783676,1.0,1,no_ausente
7531,2024-01-20,1783676,1.0,1,no_ausente
8165,2024-01-26,1783676,1.0,1,no_ausente


In [8]:
# This matrix can be ellaborated by simply pivoting our work calendar on absence_type
work_matrix = work_calendar.pivot_table(index='person_id',columns='absence_type', values='day', aggfunc='count', fill_value='')

In [9]:
# Write to CSV
work_matrix.to_csv('worker_attendance_summary.csv',index=True)

In [10]:
# Write to database
schema = 'orquest_analytics'
mydb, engine = ocu.connect_db(ocu.host, ocu.user, ocu.password, ocu.dbname, schema)
mycursor = mydb.cursor()
print('Connected to database ...')
ocu.ensure_schema_exists(schema, mycursor, mydb)

work_matrix.to_sql(name='work_matrix_python',schema=schema,con=engine, if_exists='replace', index=False)

Connected to database ...


58